# Failed attempts


### 1. rat.CellFileCollectionStack
### 2. xarray.open_mfdataset
### 3. subsetting with xarray
### 4. SwathFileCollection, rat.CellFileCollection & Julian datetype


_____________________________________

#### 1. rat.CellFileCollectionStack

Overview:

- https://ascat.readthedocs.io/en/latest/xarray_readers_tutorial.html#working-with-collections-of-cell-files
- as recommended I tried the worklow seen in the linke above
- this includes the function rat.CellFileCollectionStack.from_product_id
- as the kernel always crashed I tried to reduce the frame for the function
- for the bounding box I went from Mozambique to Buzi
- I went from couple of days to one day but it didn't work

In [ ]:
#bounding box Mozambique
bounds = (-27,-10,30,41) #latmin, latmax, lonmin, lonmax
#bounding box Buzi
bounds = (-21,-20,34,35) #latmin, latmax, lonmin, lonmax


#The timeframe we choose for now is a random date until the end of the dataset
random_date= np.datetime64("2024-05-30 20:07:09.816000")
#last_date (Last observation date): 2024-05-31 20:07:09.816000 
dates = (random_date, np.datetime64(last_date))

# Now apply the time range filter
dates = (random_date, np.datetime64(last_date))
output_bbox_filtered_by_time = output_bbox.sel(time=slice(dates[0], dates[1]))

output_bbox_filtered_by_time

output_bbox = cell_collection.read(bbox=bounds, date_range=dates)
output_bbox



### 2. xarray.open_mfdataset

Overview:

- After several tries of implementing the rat.CellFileCollectionStack.from_product_id function
- I tried a different approach with the function xarray.open_mfdataset
- the open_mfdataset function did not work and I wanted to make sure that there is no key discrepancy
- the function would just not work. Also Clay confirmed this for me

In [ ]:
# Path to the directory
directory_path = "/home/jlinke/shares/exchange/students/julian/ASCAT_Mozambique_data/"

# List all files in the directory
files = os.listdir(directory_path)

print(files)

# Loop through the files
for file in files:
    
    ncsource= f"/home/jlinke/shares/exchange/students/julian/ASCAT_Mozambique_data/{file}"
    ds=nc.Dataset(ncsource)
    list=ds.variables.keys()
    print(list)
    





### 3. subsetting with xarray

Overview:

- the core idea is to create a subset for each netCDF file that has the same frame
- the frame conists of a fixed time (f.e. a week and aligning with the ground data)
- and the bounding box for Mozambique
- the chunk size we use for the dask array is randomly chosen and seems to work
- the error occured with xrds.sel(time=... (later on in the process I learn the raw datetime type is Julian, so maybe that created and issue here as well)
- I stopped continuing this process because the research group directed me somewhere else

In [ ]:
#the core idea is to create a subset for each netCDF file that has the same frame
#the frame conists of a fixed time (f.e. a week and aligning with the ground data)
#and the bounding box for Mozambique
#The chunk size we use for the dask array is randomly chosen and seems to work
xrds=xr.open_dataset("/home/jlinke/shares/exchange/students/julian/ASCAT_Mozambique_data/1524.nc",
                     chunks={"time":100})

xrds
#at this point the question probably is weither to use the netCDF4 or the xarray library in order to take a look at the metainformation
xrds.attrs
xrds['time'].attrs
xrds.time[-1].values
#from above we know the min,max,lon,lat values for file 1524.nc
'''
Max latitude: -25.000368 Min latitude: -29.997095 

Max longitude: 34.027664 Min longitude: 30.000797'
'''
min_lat=xrds.lat.min().values
max_lat=xrds.lat.max().values
min_lon=xrds.lon.min().values
max_lon=xrds.lon.max().values

print(min_lat)
print(max_lat)
print(min_lon)
print(max_lon)

#It looks like there are NaNs in our data that we have to filter
fill_value = 9.96921e+36  # Known fill value

# Replace fill values with NaNs
xrds['lat'] = xrds['lat'].where(xrds['lat'] != fill_value, np.nan)
xrds['lon'] = xrds['lon'].where(xrds['lon'] != fill_value, np.nan)

#now dropping the nan value
xrds.dropna(dim="locations")

# Now compute min/max safely
min_lat = xrds.lat.min().values
max_lat = xrds.lat.max().values
min_lon = xrds.lon.min().values
max_lon = xrds.lon.max().values

print(min_lat, max_lat, min_lon, max_lon)

xrds.coords
#At this point we create a subset of our xarray for the area of Mozambique and timerange tx

#tx=[np.datetime64('2024-03-01T20:07:09.816000000'),np.datetime64('2024-03-07T20:07:09.816000000')]
subset = xrds.sel(time=('2024-03-01T20:07:09.816000000', '2024-03-07T20:07:09.816000000'), lat=(min_lat, max_lat), lon=(min_lon, max_lon))


### 4. SwathFileCollection, rat.CellFileCollection & Julian datetype

Overview:

- After figuring having troubles working with rat.CellFileCollectionStack Clay advised my to try implementing rat.CellFileCollection
- tried to create a swath collection SwathFileCollection
- in order to use the function swath_collection.stack
- to finally use CellFileCollection
- lot's of trouble shooting trying to implement SwathFileCollection 
- I figured out that the ascat data I got from Pavan seemingly has Julian dates (which h5py revealed coz it's supposedly working with the rawer data compared to netCDF4 and xarray which both showed me datetime64 in the metadata of the netCDF files ) 
- my guess was that SwathFileCollection only reads datetime64
- that's why I tried to change the datetype of the files to datetime64 without reading in the data
- but in the end I had the same error that the function doesn't find anything inside my timerange

https://ascat.readthedocs.io/en/latest/xarray_readers_tutorial.html#working-with-collections-of-cell-files

In [ ]:
#here I found out the the date type is actually Julian 

import h5py

# Open one of your NetCDF files
filename = '/home/jlinke/shares/exchange/students/julian/ASCAT_Mozambique_data/1524.nc'  # Replace with your actual file path
with h5py.File(filename, 'r') as f:
    # List all datasets to see their names
    print("Datasets in the file:", list(f.keys()))
    
    # Check the time variable
    if 'time' in f:
        time_data = f['time'][:]
        print("Time data:", time_data)
        print("Time data type:", time_data.dtype)





#this way you can change the date type of the xarray that is loaded into the memory
#this way we can change the date type without actually loading the data
#but it eventually doesn't change the locally saved files that I would need for function SwathFileCollection

import xarray as xr

# Open the dataset (only metadata is loaded)
source = "/home/jlinke/shares/exchange/students/julian/ASCAT_Mozambique_data/1524.nc"
ds = xr.open_dataset(source, decode_times=False)

# Check the 'time' variable
print("Original time variable:", ds['time'])

# Convert the time variable to datetime64 using xarray's built-in decoding (without loading the data)
# Seemingly the actual default mode for the xarray time unit is nanoseconds

ds['time'] = ds['time'].astype('datetime64[ns]')

# Now the time variable is in datetime64 format
print("Converted time variable:", ds['time'])

# Check the values to see the datetime64 format (it will be loaded into memory only when accessed)
print(ds['time'].values)






#with this function I can change the date type from Julian to datetime64 permanently 
#without loading the data into memory

import xarray as xr
import os
import glob

# Define the input directory and output directory
input_dir = "/home/jlinke/shares/exchange/students/julian/ASCAT_Mozambique_data/"
output_dir = "/home/jlinke/shares/exchange/students/julian/ASCAT_Mozambique_data/xr_to_netcdf/"

# Ensure the output directory exists
os.makedirs(output_dir, exist_ok=True)

# Find all NetCDF files in the input directory
netcdf_files = glob.glob(os.path.join(input_dir, "*.nc"))

# Loop through each file
for file_path in netcdf_files:
    print(f"Processing {file_path}...")
    
    # Open the dataset without decoding time automatically
    ds = xr.open_dataset(file_path, decode_times=False)
    
    # Convert time variable to datetime64 (if 'time' exists in the dataset)
    if 'time' in ds.variables:
        ds['time'] = ds['time'].astype('datetime64[ns]')
        
        # Remove conflicting attributes (units and calendar)
        ds['time'].attrs.pop('units', None)
        ds['time'].attrs.pop('calendar', None)

    # Define output file path
    output_file = os.path.join(output_dir, os.path.basename(file_path))

    # Save the modified dataset
    ds.to_netcdf(output_file)
    print(f"Saved modified dataset to {output_file}")

print("Processing complete for all files.")





#a quick check if the time conversion worked

import h5py

# Open one of your NetCDF files
filename = '/home/jlinke/shares/exchange/students/julian/ASCAT_Mozambique_data/xr_to_netcdf/modified_1524.nc'  # Replace with your actual file path
with h5py.File(filename, 'r') as f:
    # List all datasets to see their names
    print("Datasets in the file:", list(f.keys()))
    
    # Check the time variable
    if 'time' in f:
        time_data = f['time'][:]
        print("Time data:", time_data)
        print("Time data type:", time_data.dtype)





#so here I tried rat.SwathFileCollection with the files that now have datetime64 as date type
#but I had the same error as with the original files

import julian
import datetime

swath_source="/home/jlinke/shares/exchange/students/julian/ASCAT_Mozambique_data/xr_to_netcdf/"
swath_collection = rat.SwathFileCollection.from_product_id(swath_source, "H121_V1.0")

dates = [np.datetime64("1960-01-01 01:00:00.000000"), np.datetime64("2030-01-01 01:00:00.000000")]


bounds=(-27,-10,30,41)

output = swath_collection.read(bbox=bounds, date_range=dates)





#this is how further functions would be implemented but I need further help from Clay to continue

# without setting this variable as False, the package will warn the user and wait for confirmation before running, since a careless use of `stack` pointing to the wrong directory could delete or ruin a lot of data.
rat.process_warnings = False
# where to save the files
cell_file_directory = "/home/jlinke/shares/exchange/students/julian/ASCAT_Mozambique_data/cell"
# a list of swath file names to write, if you have a specific list
fnames = None
# the date range to stack data from
date_range = (np.datetime64(datetime(2024, 5, 25)), np.datetime64(datetime(2024, 5, 31)))

# mode : "w" for creating new files if any already exist, "a" to append data to existing cell files
# note that old data and new data will not be sorted after the append
mode = "w"
# the number of processes to use when writing the data.
# does NOT have anything to do with xarray's dask processing
# I've found that using too many processes, even on machines with many cores, may not be optimal.
# A good number is 8.
processes = 8

# the maximum size of the data buffer before dumping to file (actual maximum memory used will be higher)
# default is 6144MB
buffer_memory_mb = None

# This command commented to avoid accidental execution
swath_collection.stack(
    output_dir=cell_file_directory,
    fnames=fnames,
    date_range=date_range,
    mode=mode,
    processes=processes,
    buffer_memory_mb=buffer_memory_mb
    )
#ascat.read_native.ragged_array_ts

